In [32]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import os, pickle, random
from datetime import datetime

In [33]:
get_after_QA = lambda s: s.split("QA_", 1)[1]

In [34]:
dati = pd.read_excel('/Users/lapotinacci/thesis/metric_datasets/NO2_dataset.xlsx')
dati = dati[dati["serviceUri"] == "http://www.disit.org/km4city/resource/iot/orionUNIFI/DISIT/ARPAT_QA_AR-ACROPOLI"].copy()
sensor_name = get_after_QA(dati["serviceUri"].iloc[0])
dati.drop(columns=["serviceUri"], inplace=True)
print(dati.shape)

(308, 16)


In [35]:
# === Creazione cartella esperimento con timestamp ISO ===
timestamp = datetime.now().strftime("%H-%M-%S")
print(f"Ora inizio esperimento: {timestamp}")
day= datetime.now().strftime("%Y-%m-%d")
print(f"Giorno inizio esperimento: {day}")
exp_dir = f"../RISULTATI/{day}/esperimento_{sensor_name}_{timestamp}"

os.makedirs(exp_dir, exist_ok=True)

print(f"Cartella esperimento creata: {exp_dir}")

Ora inizio esperimento: 14-19-01
Giorno inizio esperimento: 2025-11-30
Cartella esperimento creata: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01


In [36]:
import ast

mask = ~dati['TTT'].str.contains("nan", na=True)
dtset_completo = dati[mask]

dtset_completo['TTT'] = dtset_completo['TTT'].apply(ast.literal_eval)

In [37]:
# Filtra le righe dove la colonna TTT ha lunghezza 24, ed elimina tutte le colonne le cui osservazioni sono indipendenti dal giorno
dtset_filtrato = dtset_completo[dtset_completo["TTT"].apply(lambda x: len(x) == 24)].sample(frac=1).reset_index(drop=True)
dtset_filtrato = dtset_filtrato[dtset_filtrato["type_of_TTT"] == "daily"]
len(dtset_filtrato)

272

In [38]:
import numpy as np


def is_useful_series(series, zero_ratio_thr=0.85, min_nonzero_value=0.1):
    arr = np.array(series, dtype=float)
    
    # Serie troppo corta
    if len(arr) <= 1:
        return False
    
    # Troppi zeri
    zero_ratio = (arr == 0).mean()
    if zero_ratio >= zero_ratio_thr:
        return False
    
    # Valore medio trascurabile (quasi zero)
    mean_val = np.mean(arr)
    if abs(mean_val) < min_nonzero_value:
        return False
    
    # Se arriva qui, la serie ha valori significativi
    return True

# Applica filtro
mask_useful = dtset_filtrato["TTT"].apply(is_useful_series)
df_useful = dtset_filtrato[mask_useful]

In [39]:
def spezza_serie_in_colonne(df, colonna_stringa, prefix='col_'):
    # --- Controllo colonna ---
    if colonna_stringa not in df.columns:
        raise ValueError(f"La colonna '{colonna_stringa}' non esiste nel DataFrame")

    result_df = df.copy()

    # Estraggo la colonna contenente gli embedding
    col = result_df.pop(colonna_stringa)

    embeddings = []

    for item in col:
        if pd.isna(item) or item == "":
            embeddings.append([])
            continue
        
        try:
            # Parsing sicuro della stringa in lista Python
            vett = ast.literal_eval(item)

            # Se l’embedding è una numpy array o altro iterabile, lo converto in list
            if not isinstance(vett, list):
                vett = list(vett)

            # Converto tutto in float
            vett = [float(x) for x in vett]

            embeddings.append(vett)

        except Exception as e:
            print(f"Errore nel parsing dell'elemento '{item}': {e}")
            embeddings.append([])

    # --- Uniformo dimensione dei vettori ---
    lengths = [len(v) for v in embeddings]
    if not lengths:
        raise ValueError("Nessun embedding valido nella colonna")

    max_len = max(lengths)

    embeddings_padded = [
        v + [np.nan] * (max_len - len(v)) if len(v) < max_len else v
        for v in embeddings
    ]

    # --- Creo DataFrame delle nuove colonne ---
    colonne_embedding = pd.DataFrame(
        embeddings_padded,
        index=result_df.index,
        columns=[f"{prefix}{i}" for i in range(max_len)]
    )

    # --- Merge col DataFrame originale ---
    return pd.concat([result_df, colonne_embedding], axis=1)

In [40]:
for ttt in df_useful.TTT:
    plt.plot(ttt)

In [41]:
def summary_df(df):
    summary = pd.DataFrame({
        'dtype': df.dtypes,
        'n_missing': df.isna().sum(),
        '%_missing': (df.isna().mean() * 100).round(2),
        'n_unique': df.nunique(),
        'top_value': df.mode().iloc[0],  # moda
        'top_freq': [df[col].value_counts(dropna=True).iloc[0] if not df[col].value_counts(dropna=True).empty else None for col in df.columns],
        'min': [df[col].min() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
        'max': [df[col].max() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
        'mean': [df[col].mean() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
        'std': [df[col].std() if pd.api.types.is_numeric_dtype(df[col]) else None for col in df.columns],
    })
    return summary


In [42]:
#Nella precedente sono state eliminate le colonne con Nan, tuttavia alcuni dataset dei sensori METRO hanno dei Nan in city e surface dunque vengono rimossi nel prcedente box

categorical = [
    'is_weekend'
    ]

#consiglio, per serie cicliche usare funzioni periodiche tipo seno coseno 
df_encoded = pd.get_dummies(df_useful
                            , columns=categorical)


In [43]:
def save_pkl(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
    print(f"Salvato: {path}")

def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

In [44]:
#codifica ciclica delle features e individuazione del'ora del giorno e della settimana 

def encode_time_features(df, start_col="interval_start", end_col="interval_end"):
    df = df.copy()
    
    # Step 1: Ensure datetime columns are in the correct format first
    df[start_col] = pd.to_datetime(df[start_col], utc=True, format="mixed")
    df[end_col] = pd.to_datetime(df[end_col], utc=True, format="mixed")

    # Step 2: Extract 'month' and 'day' after conversion
    df['month'] = df[start_col].dt.month
    df['day'] = df[start_col].dt.dayofweek # Note: dayofweek is typically used for cyclical weekly patterns
    
    # Step 3: Now you can create the cyclical features
    df['month_sin'] = np.sin(2 * np.pi * df['month']/len(df['month'].unique()))
    df['month_cos'] = np.cos(2 * np.pi * df['month']/len(df['month'].unique()))
    df['day_sin'] = np.sin(2 * np.pi * df['day']/len(df['day'].unique()))
    df['day_cos'] = np.cos(2 * np.pi * df['day']/len(df['day'].unique()))


    # Ora del giorno (0-23) → ciclo giornaliero
    df["hour"] = df[start_col].dt.hour
    angle_day_start = 2 * np.pi * df["hour"] / 24
    df["start_time_sin"] = np.sin(angle_day_start)
    df["start_time_cos"] = np.cos(angle_day_start)

    df["hour_end"] = df[end_col].dt.hour
    angle_day_end = 2 * np.pi * df["hour_end"] / 24
    df["end_time_sin"] = np.sin(angle_day_end)
    df["end_time_cos"] = np.cos(angle_day_end)

    # Durata in minuti
    df["duration_minutes"] = (df[end_col] - df[start_col]).dt.total_seconds() / 60

    # Giorno della settimana (0=Mon..6=Sun) → ciclo settimanale
    dayofweek = df[start_col].dt.dayofweek
    angle_week = 2 * np.pi * dayofweek / 7
    df["dow_sin"] = np.sin(angle_week)
    df["dow_cos"] = np.cos(angle_week)

    # Giorno dell’anno (1-365) → ciclo annuale
    dayofyear = df[start_col].dt.dayofyear
    angle_year = 2 * np.pi * dayofyear / 365
    df["doy_sin"] = np.sin(angle_year)
    df["doy_cos"] = np.cos(angle_year)

    # Rimuovo colonne temporali e variabili ausiliarie
    df = df.drop(columns=[start_col, end_col, "hour", "hour_end", "day", "month"])
    return df

embedding_cols = ['highway_embedding', 'AreaTypeEmbeddings']

df_final = encode_time_features(df_encoded)
df_final.drop(columns=['type_of_TTT'], inplace=True)
for emb_col in embedding_cols:
    df_final = spezza_serie_in_colonne(df_final, emb_col, prefix=f'{emb_col}_')
df_final.shape

# Salvataggio DataFrame finale
df_path = os.path.join(exp_dir, f"dataframe_{sensor_name}")
os.makedirs(df_path, exist_ok=True)
save_pkl(df_final, os.path.join(df_path, f"filtered_dataframe.pkl"))


Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/dataframe_AR-ACROPOLI/filtered_dataframe.pkl


In [45]:
for ttt in df_final.TTT[10:20]:
    plt.plot(ttt)
    plt.title("Esempi di TTT ARPAT")
    plt.xlabel("Ora del Giorno")
    plt.ylabel("Valore TTT")

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def series_to_2d_array(series, output_dim=24):
    arr = np.stack(series.apply(lambda x: np.array(x, dtype=np.float32)))
    if arr.shape[1] != output_dim:
        raise ValueError(f"Ogni elemento deve avere {output_dim} valori, trovato {arr.shape[1]}")
    return arr

 # Separazione delle feature e del target
X = df_final.drop(columns=['TTT'])
y = df_final['TTT']

X_np = X.to_numpy().astype(np.float32)
y_np = series_to_2d_array(y, output_dim=24)

# Calcolo deviazione standard per ogni serie
std_scores = np.std(y_np, axis=1)

std_scores_log = np.log1p(std_scores)  # log(1+x) per evitare problemi con valori vicini a 0

std_scores_normalized = (std_scores_log - np.min(std_scores_log)) / (np.max(std_scores_log) - np.min(std_scores_log))

# Creo bin basati sui quantili
bins = np.array([0.0, 0.4, 0.8, 1.0])
std_bins = np.digitize(std_scores_normalized, bins)

# Primo split: train (80%) vs temp (20%)
# Passo anche std_bins per poter stratificare
X_train, X_test, y_train, y_test = train_test_split(
    X_np, y_np,
    test_size=0.2,
    random_state=42,
    stratify=std_bins, 
    shuffle=True
)

print(f"Train size: {X_train.shape}, {y_train.shape}")
print(f"Test size: {X_test.shape}, {y_test.shape}")

# Debug prints
print(f"Final shapes:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

# Scaler per input
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test  = scaler_X.transform(X_test)

# Scaler per output (se serve normalizzare anche y)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test  = scaler_y.transform(y_test)

#cartella salvataggio train e test
path = os.path.join(exp_dir, "train_test_data")
os.makedirs(path, exist_ok=True)

#  === Salvataggio dataset e scaler in formato .pkl ===
save_pkl(scaler_X.inverse_transform(X_train), os.path.join('/Users/lapotinacci/thesis/Federated_Sys/train_test', f"X_train_unscaled_{sensor_name}.pkl"))
save_pkl(scaler_y.inverse_transform(y_train), os.path.join('/Users/lapotinacci/thesis/Federated_Sys/train_test', f"y_train_unscaled_{sensor_name}.pkl"))
save_pkl(scaler_X.inverse_transform(X_test), os.path.join('/Users/lapotinacci/thesis/Federated_Sys/train_test', f"X_test_unscaled_{sensor_name}.pkl"))
save_pkl(scaler_y.inverse_transform(y_test), os.path.join('/Users/lapotinacci/thesis/Federated_Sys/train_test', f"y_test_unscaled_{sensor_name}.pkl"))

save_pkl(X_train, os.path.join(exp_dir, "X_train.pkl"))
save_pkl(y_train, os.path.join(exp_dir, "y_train.pkl"))
save_pkl(X_test, os.path.join(exp_dir, "X_test.pkl"))
save_pkl(y_test, os.path.join(exp_dir, "y_test.pkl"))

save_pkl(scaler_X, os.path.join(exp_dir, "scaler_X.pkl"))
save_pkl(scaler_y, os.path.join(exp_dir, "scaler_y.pkl"))

print("Scaler salvati.")

print("\nTutti i file sono stati salvati correttamente.")
print(f"Percorso cartella: {os.path.abspath(exp_dir)}")

Train size: (217, 33), (217, 24)
Test size: (55, 33), (55, 24)
Final shapes:
X_train: (217, 33)
y_train: (217, 24)
Salvato: /Users/lapotinacci/thesis/Federated_Sys/train_test/X_train_unscaled_AR-ACROPOLI.pkl
Salvato: /Users/lapotinacci/thesis/Federated_Sys/train_test/y_train_unscaled_AR-ACROPOLI.pkl
Salvato: /Users/lapotinacci/thesis/Federated_Sys/train_test/X_test_unscaled_AR-ACROPOLI.pkl
Salvato: /Users/lapotinacci/thesis/Federated_Sys/train_test/y_test_unscaled_AR-ACROPOLI.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/X_train.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/y_train.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/X_test.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/y_test.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/scaler_X.pkl
Salvato: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/scaler_y.pkl
Scaler salvati.

Tutti i file sono stati salvat

In [47]:
# === Costruisci i path ===
path_X_train = os.path.join(exp_dir, "X_train.pkl")
path_y_train = os.path.join(exp_dir, "y_train.pkl")
path_X_test  = os.path.join(exp_dir, "X_test.pkl")
path_y_test  = os.path.join(exp_dir, "y_test.pkl")

path_scaler_X = os.path.join(exp_dir, "scaler_X.pkl")
path_scaler_y = os.path.join(exp_dir, "scaler_y.pkl")

# === Ricaricamento ===
X_train = load_pkl(path_X_train)
y_train = load_pkl(path_y_train)
X_test  = load_pkl(path_X_test)
y_test  = load_pkl(path_y_test)


scaler_X = load_pkl(path_scaler_X)
scaler_y = load_pkl(path_scaler_y)

print("Shapes ricostruite:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

print("\nScaler caricati:")
print("scaler_X:", type(scaler_X))
print("scaler_y:", type(scaler_y))


Shapes ricostruite:
X_train: (217, 33)
y_train: (217, 24)
X_test: (55, 33)
y_test: (55, 24)

Scaler caricati:
scaler_X: <class 'sklearn.preprocessing._data.StandardScaler'>
scaler_y: <class 'sklearn.preprocessing._data.StandardScaler'>


# modelli

In [48]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers # type: ignore
from tensorflow.keras.layers import Dropout # type: ignore
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [49]:
def save_prediction_plots(y_true, y_pred, output_dir, prefix="pred_plot"):
    os.makedirs(output_dir, exist_ok=True)

    n = len(y_true)
    print(f"Salvo {n} plot in: {output_dir}")

    for i in range(n):
        plt.figure(figsize=(8, 3))
        plt.plot(y_true[i], label="True")
        plt.plot(y_pred[i], label="Pred")
        plt.legend()
        plt.tight_layout()

        filename = f"{prefix}_{i+1}.png"
        path = os.path.join(output_dir, filename)

        plt.savefig(path)
        plt.close()

    print("Plot salvati correttamente.")


In [50]:
# Definizione modello DNN
def build_model(input_shape: tuple, output_shape: tuple):
    model = models.Sequential([
        layers.Dense(128, activation = "relu", input_shape=input_shape),
        Dropout(0.0),
        layers.Dense(output_shape, 'linear')
    ])
    model.compile(
    optimizer=optimizers.AdamW(learning_rate=0.0101, weight_decay=0.0107),
    loss= 'mse',
    metrics=['mae'],
    )
    return model

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',      # metrica da monitorare, es. val_loss o val_mae
    patience=100,             # numero di epoche senza miglioramento prima di fermare
    restore_best_weights=True,  # ripristina i pesi migliori trovati
    verbose = 1
)

# Training del modello
model = build_model((X_test.shape[1],), y_test.shape[1])
history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=256,
    callbacks = [early_stop],
    validation_split = 0.2,
    verbose=1
    )

'''print("y_train min/max:", y_train_np.min(), y_train_np.max())
print("y_pred min/max (scaled):", y_pred.min(), y_pred.max())
y_pred_rescaled = scaler_y.inverse_transform(y_pred)
print("y_pred_rescaled min/max:", y_pred_rescaled.min(), y_pred_rescaled.max())'''

if not os.path.exists(os.path.join(exp_dir, "model")):
    os.makedirs(os.path.join(exp_dir, "model"))
    model.save(os.path.join(exp_dir, "model", "NO2_MAREMMA_model.keras"))


Epoch 1/1000


/Users/lapotinacci/thesis/Federated_Sys/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step - loss: 1.3128 - mae: 0.9042 - val_loss: 0.9736 - val_mae: 0.7270
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.9488 - mae: 0.7620 - val_loss: 0.8203 - val_mae: 0.6688
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.7618 - mae: 0.6783 - val_loss: 0.7321 - val_mae: 0.6380
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.6420 - mae: 0.6201 - val_loss: 0.6693 - val_mae: 0.6139
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.5598 - mae: 0.5776 - val_loss: 0.6168 - val_mae: 0.5968
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.5040 - mae: 0.5493 - val_loss: 0.5687 - val_mae: 0.5807
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4643 - mae: 0.5296 - val_loss: 0.5259 - val_mae: 0.5630
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.4326 - mae: 0.5131 - val_loss: 0.4908 - val_mae: 0.5477
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.4033 - mae: 0.4

In [51]:
model = tf.keras.models.load_model(f'{exp_dir}/model/NO2_{sensor_name}_model.keras') if os.path.exists(f'{exp_dir}/model/NO2_{sensor_name}_model.keras') else model

In [52]:
# Estrai losses da tutti i folds
train_losses = history.history['loss']
val_losses   = history.history['val_loss']

train_losses = train_losses
val_losses = val_losses

# Plot
plt.figure(figsize=(8,5))


# Training loss
plt.plot(train_losses, color='blue', label='Train Loss')
# Validation loss
plt.plot(val_losses, color='red', label='Validation Loss')


# Dettagli grafico
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.savefig(f'{exp_dir}/loss_plot.png')



In [53]:
def plot_first_n_series(y_pred, y_test, n_series=10, title="Confronto Serie Predette vs Reali"):
    """
    Plotta le prime n_series serie reali e predette per un confronto diretto.

    Args:
        y_pred (np.ndarray): Predizioni del modello (shape: num_samples x seq_len).
        y_test (np.ndarray): Serie reali (shape: num_samples x seq_len).
        n_series (int): Numero di serie da plottare.
        title (str): Titolo del grafico.
    """

    # Limitiamo a n_series per evitare problemi
    n = min(n_series, y_pred.shape[0], y_test.shape[0])

    plt.figure(figsize=(14, 8))

    for i in range(n):
        plt.plot(
            y_test[i], 
            linewidth=1.8, 
            alpha=0.5, 
            label=f"Reale {i+1}" if i == 0 else "",
            color = 'blue'
        )
        plt.plot(
            y_pred[i], 
            linewidth=1.2, 
            linestyle="--", 
            alpha=0.9, 
            label=f"Predetto {i+1}" if i == 0 else "",
            color = 'orange'
        )

    plt.title(title, fontsize=16)
    plt.xlabel("Time Step", fontsize=12)
    plt.ylabel("Valore", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [54]:
import seaborn as sns

def plot_diagnosis_gradient(y_pred, y_test, save_path: str,  tolerance:float =0.05, max_samples_heatmap=50, scatter_sample_ratio=0.5, tol_auto = True):
    """
    Dashboard diagnostica con tolleranza AUTO-ADATTIVA alla scala dei dati.
    
    Args:
        tolerance (float): 
            - Se None: La tolleranza viene calcolata automaticamente come il 5% del range dei dati (Max-Min).
            - Se float: Valore assoluto (es. 0.5).
        tol_auto (bool): Se True, usa la tolleranza automatica basata sul range dei dati, altrimenti usa il valore manuale fornito 
        col parametro tolerance.
    """
    
    y_test_flat = y_test.flatten()
    y_pred_flat = y_pred.flatten()

    # ---------------------------------------------------------
    # AUTOMAZIONE TOLLERANZA
    # ---------------------------------------------------------
    if tol_auto:
        data_range = y_test_flat.max() - y_test_flat.min()
        # Impostiamo la tolleranza al 5% del range totale (regolabile)
        tolerance = data_range * tolerance 
        tol_label = f"Auto (5% Range: ±{tolerance:.2f})"
    else:
        tol_label = f"Manuale (±{tolerance})"

    fig, axes = plt.subplots(1, 2, figsize=(22, 8))

    # -----------------------------
    # 1. HEATMAP (SX)
    # -----------------------------
    errors = np.abs(y_test - y_pred)
    errors_subset = errors[:max_samples_heatmap]
    
    # Adattiamo anche la scala colori della heatmap alla tolleranza
    # Tutto ciò che è oltre 3 volte la tolleranza è "errore massimo" visivo
    sns.heatmap(errors_subset, cmap="plasma", ax=axes[0], 
                vmax=tolerance * 3,
                cbar_kws={'label': 'Errore Assoluto'})
    
    axes[0].set_title(f"Heatmap Errori (Primi {max_samples_heatmap} campioni)", fontsize=16)
    axes[0].set_xlabel("Time Step", fontsize=12)
    axes[0].set_ylabel("Indice Campione", fontsize=12)

    # -----------------------------
    # 2. SCATTER CON SFONDO GRADIENTE (DX)
    # -----------------------------
    
    # Campionamento dati
    if scatter_sample_ratio < 1.0:
        mask = np.random.rand(len(y_test_flat)) < scatter_sample_ratio
        sample_test = y_test_flat[mask]
        sample_pred = y_pred_flat[mask]
    else:
        sample_test = y_test_flat
        sample_pred = y_pred_flat

    # Calcolo limiti grafico
    min_val = min(y_test_flat.min(), y_pred_flat.min())
    max_val = max(y_test_flat.max(), y_pred_flat.max())
    padding = (max_val - min_val) * 0.05
    plot_min = min_val - padding
    plot_max = max_val + padding

    # --- SFONDO GRADIENTE ---
    grid_x, grid_y = np.meshgrid(
        np.linspace(plot_min, plot_max, 200),
        np.linspace(plot_min, plot_max, 200)
    )
    grid_z = np.abs(grid_y - grid_x)

    im = axes[1].imshow(
        grid_z, 
        extent=(plot_min, plot_max, plot_min, plot_max), 
        origin='lower', 
        cmap='RdYlGn_r', 
        alpha=0.4, 
        vmax=tolerance * 4, # Il rosso satura a 4x della tolleranza
        aspect='auto'
    )

    # --- PUNTI BLU ---
    axes[1].scatter(
        sample_test, 
        sample_pred, 
        alpha=0.6, 
        s=15, 
        color='royalblue', 
        edgecolors='white', 
        linewidth=0.3,
        label='Campioni'
    )

    axes[1].plot([plot_min, plot_max], [plot_min, plot_max], 'k--', alpha=0.5, label='Perfetto (y=x)', color = 'red')

    # Metriche
    within_tol = np.mean(np.abs(y_test_flat - y_pred_flat) <= tolerance) * 100

    axes[1].set_title(
        f"Bontà delle previsioni con {tol_label}\nValori nel range di Tolleranza: {within_tol:.1f}%", 
        fontsize=16
    )
    axes[1].set_xlabel("Valore Reale", fontsize=12)
    axes[1].set_ylabel("Valore Predetto", fontsize=12)
    axes[1].set_xlim(plot_min, plot_max)
    axes[1].set_ylim(plot_min, plot_max)
    
    cbar = plt.colorbar(im, ax=axes[1], pad=0.02)
    cbar.set_label('Gravità Errore (Distanza da diagonale)')

    plt.tight_layout()
    plt.savefig(f'{exp_dir}/{save_path}.png')

In [55]:
y_pred = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred)
y_test_np = scaler_y.inverse_transform(y_test)

plot_diagnosis_gradient(y_pred, y_test_np, save_path='pred_quality', max_samples_heatmap=10, scatter_sample_ratio=0.7)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/stepWARNING:tensorflow:6 out of the last 8 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x125eff9c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/var/folders/1s/vnvjtlb516n5mswtv7rf58b80000gn/T/ipykernel_2013/2011710758.py:96: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  axes[1].plot([plot_min, plot_max], [plot_min, plot_max], 'k--', alpha=0.5, label='Perfetto (y=x)', color = 'red')


In [56]:
save_prediction_plots(y_test_np, y_pred, output_dir=f'{exp_dir}/TEST', prefix="pred_plot")

Salvo 55 plot in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/TEST
Plot salvati correttamente.


In [57]:
import json
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_and_log(y_true, y_pred, log_path="log.json"):
    # --- Metriche ---
    mae = mean_absolute_error(y_true, y_pred, multioutput="uniform_average")
    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput="uniform_average"))

    mask = y_true != 0
    mape = float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0)

    r2 = r2_score(y_true, y_pred, multioutput="uniform_average")

    # Valore medio del test set
    v_mean = float(np.mean(y_true))

    # --- Stampa ---
    print(f"Test MAE: {mae:.4f}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test MAPE: {mape:.2f}%")
    print(f"Test R2: {r2:.4f}")
    print(f"Valore medio del test set: {v_mean:.4f}")

    # --- Salvataggio log ---
    log_data = {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "R2": r2,
        "test_mean_value": v_mean
    }

    with open(log_path, "w") as f:
        json.dump(log_data, f, indent=4)

    print(f"\nLog salvato in: {log_path}")
evaluate_and_log(y_test_np, y_pred, log_path=f"{exp_dir}/evaluation_log.json")
# Impostazioni pandas per visualizzare tutti i dati

Test MAE: 0.0113
Test RMSE: 0.0691
Test MAPE: 0.19%
Test R2: 0.9971
Valore medio del test set: 5.9393

Log salvato in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/evaluation_log.json


# globale e federato

In [59]:
fed = tf.keras.models.load_model('/Users/lapotinacci/thesis/Federated_Sys/app/custom/models/fed_model_Arpat.keras')

model  = [fed]
model_name = ["federated"]

In [60]:
checkpoint_filepath = f'{exp_dir}/model/{sensor_name}_fed_model.keras'

# 2. Crea l'istanza della callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_weights_only=False
)

In [61]:
for model, name in zip(model, model_name):
    y_pred = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test_np = scaler_y.inverse_transform(y_test)

    print(f"--- {name} ---")
    #plot_diagnosis_gradient(y_pred, y_test_np, max_samples_heatmap=10, scatter_sample_ratio=0.7)
    evaluate_and_log(y_test_np, y_pred, log_path=f"{exp_dir}/{name.replace(' ', '_').lower()}_evaluation_log.json")
    save_prediction_plots(y_test_np, y_pred, output_dir=f'{exp_dir}/{name.replace(" ", "_").lower()}/TEST', prefix="pred_plot")

    print("\n")
    if name == 'federated':
        model.fit(X_train, y_train, epochs=1000, batch_size=64, callbacks=[model_checkpoint_callback], validation_split=0.2)
        model.load_weights(checkpoint_filepath)
        y_pred = model.predict(X_test)
        y_pred = scaler_y.inverse_transform(y_pred)
        print(f"--- {name} after local fine-tuning ---")
        plot_diagnosis_gradient(y_pred, y_test_np, save_path=f"{name}_pred_quality", max_samples_heatmap=10, scatter_sample_ratio=0.7)
        save_prediction_plots(y_test_np, y_pred, output_dir=f'{exp_dir}/{name.replace(" ", "_").lower()}_fd/TEST', prefix="pred_plot")
        evaluate_and_log(y_test_np, y_pred, log_path=f"{exp_dir}/{name.replace(' ', '_').lower()}_fd_evaluation_log.json")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
--- federated ---
Test MAE: 0.7015
Test RMSE: 0.9790
Test MAPE: 13.20%
Test R2: 0.2679
Valore medio del test set: 5.9393

Log salvato in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/federated_evaluation_log.json
Salvo 55 plot in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/federated/TEST
Plot salvati correttamente.


Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.6644 - mae: 0.6280 - mse: 0.6644 - val_loss: 0.4725 - val_mae: 0.5202 - val_mse: 0.4725
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3583 - mae: 0.4558 - mse: 0.3583 - val_loss: 0.3180 - val_mae: 0.4269 - val_mse: 0.3180
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2488 - mae: 0.3787 - mse: 0.2488 - val_loss: 0.2432 - val_mae: 0.3741 - val_mse: 0.2432
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1794 - mae: 0.3207 - mse: 0.1794 - val_loss: 0.1546 - val_mae: 0.2972 - val_mse: 0.1546
Epoch 5/1000
3/3 ━━━━

/var/folders/1s/vnvjtlb516n5mswtv7rf58b80000gn/T/ipykernel_2013/2011710758.py:96: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  axes[1].plot([plot_min, plot_max], [plot_min, plot_max], 'k--', alpha=0.5, label='Perfetto (y=x)', color = 'red')


Salvo 55 plot in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/federated_fd/TEST
Plot salvati correttamente.
Test MAE: 0.0097
Test RMSE: 0.0334
Test MAPE: 0.17%
Test R2: 0.9992
Valore medio del test set: 5.9393

Log salvato in: ../RISULTATI/2025-11-30/esperimento_AR-ACROPOLI_14-19-01/federated_fd_evaluation_log.json
